In [64]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

ModuleNotFoundError: No module named 'google'

In [ ]:
# fasttext
! pip install fasttext

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('/Users/badro97/Section3/sprint3/project/Project3/real_finish.csv')

In [ ]:
# fasttext input_form 형식으로 변형
Cat = data['category']+"_"+data['sub_category']
cats = Cat.str.replace("/"," ")
data['input_form'] = '__label__' + cats + ' ' + data['title']
df = data[['input_form']]

In [ ]:
df

,input_form
0,__label__김치_포기김치 국내산 배추 김치 포기 김장김치 주문
1,__label__김치_포기김치 피코크 조선호텔 포기김치
2,__label__김치_포기김치 대상 종가집 행복이온 포기김치
3,__label__김치_포기김치 대상 종가집 전라도 행복이온 포기김치
4,__label__김치_포기김치 선화동매운실비김치 맛있게 매운 선화동 실비 김치
...,...
205375,__label__밀키트_찌개 국 청정원 호밍스 사골 시래기 된장국
205376,__label__밀키트_찌개 국 먹템 매콤한 극락 캠핑음식 용밀 간편조리식품 키트
205377,__label__밀키트_찌개 국 연말축제 청정원 호밍스 한우진곰탕
205378,__label__밀키트_찌개 국 프레시지 간편한 직화용기 꼬치어묵탕


In [ ]:
df.to_csv('input.csv', index=False)

In [ ]:
# train : val : test = 3 : 1 : 1
train, test = train_test_split(df, test_size=0.20, shuffle=True, random_state=42)
train, val = train_test_split(train, test_size=0.25, shuffle=True, random_state=42)
train.reset_index(inplace=True)
val.reset_index(inplace=True)
test.reset_index(inplace=True)

In [ ]:
train.to_csv('train.csv', index=False, columns=['input_form'])
val.to_csv('val.csv', index=False, columns=['input_form'])
test.to_csv('test.csv', index=False, columns=['input_form'])

In [ ]:
import fasttext
from tqdm import tqdm

simple = fasttext.train_supervised(input='train.csv'
                                  ,epoch=10
                                  ,ws=5
                                  ,lr = 0.5
                                  ,wordNgrams=2
                                  ,dim=100
                                  )

Read 0M words
Number of words:  86089
Number of labels: 108
Progress: 100.0% words/sec/thread:  212695 lr:  0.000000 avg.loss:  0.110899 ETA:   0h 0m 0s


In [ ]:
print(simple.test('val.csv'))

(41074, 0.9350440668062522, 0.9350440668062522)


In [ ]:
print(simple.predict("훠궈 맛있다"))

(('__label__축산물_기타육류',), array([0.17789043]))


In [ ]:
print(simple.predict("말벡 와인 마시따"))

(('__label__전통주_과실주',), array([0.99360412]))


In [ ]:
print(simple.test('test.csv'))

(41075, 0.9377480219111382, 0.9377480219111382)


In [ ]:
print(simple.predict("해장 라면"))

(('__label__라면',), array([0.76587141]))


In [ ]:
simple.save_model("model.bin")

In [ ]:
small = fasttext.train_supervised(input='train.csv')
small.save_model("small.bin")

Read 0M words
Number of words:  86089
Number of labels: 108
Progress: 100.0% words/sec/thread:  227340 lr:  0.000000 avg.loss:  0.555702 ETA:   0h 0m 0s


In [61]:
print(small.test('test.csv'))

(41075, 0.9320024345709069, 0.9320024345709069)
